In [44]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import os  # Import the os module
import numpy as np

In [45]:
def parse_xml_to_csv(xml_file, csv_file):
    # Check if the CSV file already exists
    if not os.path.exists(csv_file):
        # Parse the XML file
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Create a CSV file and write the headers
        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            headers = ['Type', 'Source Name', 'Source Version', 'Device', 'Unit', 'Creation Date', 'Start Date', 'End Date', 'Value']
            writer.writerow(headers)

            # Iterate through each Record element in the XML
            for record in root.findall('.//Record'):
                type = record.get('type')
                source_name = record.get('sourceName')
                source_version = record.get('sourceVersion')
                device = record.get('device')
                unit = record.get('unit')
                creation_date = record.get('creationDate')
                start_date = record.get('startDate')
                end_date = record.get('endDate')
                value = record.get('value')

                # Write the record data to the CSV file
                writer.writerow([type, source_name, source_version, device, unit, creation_date, start_date, end_date, value])
    else:
        print(f"CSV file '{csv_file}' already exists. Skipping XML parsing.")


In [46]:
df = pd.read_csv('output_health_data.csv')

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1917706809.py:1: DtypeWarning: Columns (2,3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('output_health_data.csv')


In [47]:
def sleep_session_summary(data):
    # Load the CSV data into a DataFrame
    df = data
    
    # Filter the DataFrame for sleep records that are not 'InBed'
    sleep_df = df[(df['Type'].str.contains('Sleep')) & (~df['Value'].astype('str').str.contains('HKCategoryValueSleepAnalysisInBed'))]
    sleep_df['Start DateTime'] = pd.to_datetime(sleep_df['Start Date'])
    sleep_df['End DateTime'] = pd.to_datetime(sleep_df['End Date'])
    
    # Sort by 'Start DateTime' to ensure correct grouping
    sleep_df = sleep_df.sort_values('Start DateTime')
    
    # Detect gaps between sleep sessions (e.g., more than an hour difference)
    sleep_df['gap'] = (sleep_df['Start DateTime'] - sleep_df['End DateTime'].shift()).dt.total_seconds() > 3600
    sleep_df['session_id'] = sleep_df['gap'].cumsum()
    
    # Group by 'session_id' and calculate the total duration, start, and end times
    grouped = sleep_df.groupby('session_id')
    summary_df = grouped.agg({
        'Start DateTime': 'first',
        'End DateTime': 'last'
    })
    summary_df['Duration'] = (summary_df['End DateTime'] - summary_df['Start DateTime']).dt.total_seconds() / 3600  # Duration in hours
    
    # return summary_df
    hr_df = df[df['Type'] == 'HKQuantityTypeIdentifierHeartRateVariabilitySDNN']
    hr_df['Start DateTime'] = pd.to_datetime(hr_df['Start Date'])
    hr_df['End DateTime'] = pd.to_datetime(hr_df['End Date'])
    hr_df['Value'] = hr_df['Value'].astype(float)
    
    # Function to calculate average HRV within the session interval
    def calculate_hrv_average(row):
        mask = (hr_df['Start DateTime'] >= row['Start DateTime']) & (hr_df['End DateTime'] <= row['End DateTime'])
        return hr_df.loc[mask, 'Value'].mean()
    
    summary_df['Average HRV (ms)'] = summary_df.apply(calculate_hrv_average, axis=1)
    
    # Rename columns for clarity
    summary_df.columns = ['Session Start', 'Session End', 'Total Sleep Duration (hrs)', 'Average HRV (ms)']
    # Add an index column based on the date of the 'Session End'
    summary_df['Date'] = summary_df['Session End'].dt.date
    summary_df['Date'] = pd.to_datetime(summary_df['Date'])
    summary_df.reset_index(inplace=True)
    summary_df.drop(columns=['session_id'], inplace=True)
    
    return summary_df

session_data = sleep_session_summary(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/4028280883.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['Start DateTime'] = pd.to_datetime(sleep_df['Start Date'])
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/4028280883.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['End DateTime'] = pd.to_datetime(sleep_df['End Date'])
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/4028280883.py:27: SettingWithCopyWarning: 
A value is tryin

In [48]:
def summarize_sleep_cycles(data):
    # Load the CSV data into a DataFrame
    df = data
    # Filter the DataFrame for sleep records
    sleep_df = df[df['Type'].str.contains('Sleep')]
    sleep_df['Start DateTime'] = pd.to_datetime(sleep_df['Start Date'])
    sleep_df['End DateTime'] = pd.to_datetime(sleep_df['End Date'])
    # Calculate sleep duration in hours
    sleep_df['Duration'] = (sleep_df['End DateTime'] - sleep_df['Start DateTime']).dt.total_seconds() / 3600
    
    # Initialize DataFrame to store sleep stages durations
    sleep_stages = pd.DataFrame()

    # Calculate duration for each sleep stage and merge into sleep_stages DataFrame
    for stage in ['Awake', 'REM', 'Core', 'Deep']:
        stage_df = sleep_df[sleep_df['Value'].astype('str').str.contains(stage)]
        stage_duration = stage_df.groupby(stage_df['Start DateTime'].dt.date)['Duration'].sum().reset_index()
        stage_duration.columns = ['Date', f'{stage} Duration (hrs)']
        if sleep_stages.empty:
            sleep_stages = stage_duration
        else:
            sleep_stages = sleep_stages.merge(stage_duration, on='Date', how='left')

    # Convert 'Date' column to datetime
    sleep_stages['Date'] = pd.to_datetime(sleep_stages['Date'])

    return sleep_stages

sleep_data = summarize_sleep_cycles(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1209704554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['Start DateTime'] = pd.to_datetime(sleep_df['Start Date'])
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1209704554.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['End DateTime'] = pd.to_datetime(sleep_df['End Date'])
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1209704554.py:9: SettingWithCopyWarning: 
A value is trying

In [49]:
def summarize_daily_steps(data):
    # Load the CSV data into a DataFrame
    df = data
    steps_df = df[df['Type'] == 'HKQuantityTypeIdentifierStepCount']
    
    # Convert the 'Start Date' to datetime format, extracting just the date part
    steps_df['Date'] = pd.to_datetime(steps_df['Start Date']).dt.date
    daily_steps = steps_df.groupby('Date')['Value'].sum().reset_index()
    daily_steps.columns = ['Date', 'Total Steps']
    standing_df = df[df['Type'] == 'HKQuantityTypeIdentifierAppleStandTime']
    standing_df['DateTime'] = pd.to_datetime(standing_df['Start Date'])
    standing_df['Hour'] = standing_df['DateTime'].dt.floor('H')  # Round down to the nearest hour
    daily_standing = standing_df.groupby(standing_df['DateTime'].dt.date)['Hour'].nunique().reset_index()
    daily_standing.columns = ['Date', 'Total Standing Hours']
    daily_steps = daily_steps.merge(daily_standing, on='Date', how='left')
    return daily_steps

# Summarize daily steps from the CSV file
daily_steps = summarize_daily_steps(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/2869529524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steps_df['Date'] = pd.to_datetime(steps_df['Start Date']).dt.date
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/2869529524.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standing_df['DateTime'] = pd.to_datetime(standing_df['Start Date'])
/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/2869529524.py:12: FutureWarning: 'H' is deprecated and w

In [50]:
def summarize_daily_basal_energy(data):
    # Load the CSV data into a DataFrame
    df = data
    basal_df = df[df['Type'] == 'HKQuantityTypeIdentifierBasalEnergyBurned']
    
    # Convert the 'Start Date' to datetime format, extracting just the date part
    basal_df['Date'] = pd.to_datetime(basal_df['Start Date']).dt.date
    
    # Sum the basal energy burned values for each day
    daily_basal = basal_df.groupby('Date')['Value'].sum().reset_index()
    daily_basal.columns = ['Date', 'Total Basal Energy Burned (kcal)']
    
    return daily_basal
daily_basal_energy = summarize_daily_basal_energy(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/202454590.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basal_df['Date'] = pd.to_datetime(basal_df['Start Date']).dt.date


In [51]:
def summarize_daily_active_energy(data):
    # Load the CSV data into a DataFrame
    df = data
    active_energy_df = df[df['Type'] == 'HKQuantityTypeIdentifierActiveEnergyBurned']
    
    # Convert the 'Start Date' to datetime format, extracting just the date part
    active_energy_df['Date'] = pd.to_datetime(active_energy_df['Start Date']).dt.date
    
    # Sum the active energy burned values for each day
    daily_active_energy = active_energy_df.groupby('Date')['Value'].sum().reset_index()
    daily_active_energy.columns = ['Date', 'Total Active Energy Burned (kcal)']
    
    return daily_active_energy
daily_active_energy = summarize_daily_active_energy(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1476006097.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_energy_df['Date'] = pd.to_datetime(active_energy_df['Start Date']).dt.date


In [52]:
def summarize_daily_exercise_minutes(data):
    # Load the CSV data into a DataFrame
    df = data
    exercise_df = df[df['Type'] == 'HKQuantityTypeIdentifierAppleExerciseTime']
    
    # Convert the 'Start Date' to datetime format, extracting just the date part
    exercise_df['Date'] = pd.to_datetime(exercise_df['Start Date']).dt.date
    
    # Sum the exercise minutes for each day
    daily_exercise = exercise_df.groupby('Date')['Value'].sum().reset_index()
    daily_exercise.columns = ['Date', 'Total Exercise Minutes']
    
    return daily_exercise
daily_exercise = summarize_daily_exercise_minutes(df)

/var/folders/zq/d4s8lx3x4hbd472pm7xggd300000gn/T/ipykernel_20721/1845882944.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exercise_df['Date'] = pd.to_datetime(exercise_df['Start Date']).dt.date


In [53]:
session_data['Date'] = pd.to_datetime(session_data['Date'])
sleep_data['Date'] = pd.to_datetime(sleep_data['Date'])
daily_steps['Date'] = pd.to_datetime(daily_steps['Date'])
daily_basal_energy['Date'] = pd.to_datetime(daily_basal_energy['Date'])
daily_active_energy['Date'] = pd.to_datetime(daily_active_energy['Date'])
daily_exercise['Date'] = pd.to_datetime(daily_exercise['Date'])
# Join the dataframes on the 'Date' column
merged_data = session_data.merge(sleep_data, on='Date', how='outer')
final_data = merged_data.merge(daily_steps, on='Date', how='outer')
final_data = final_data.merge(daily_basal_energy, on='Date', how='outer')
final_data = final_data.merge(daily_active_energy, on='Date', how='outer')
final_data = final_data.merge(daily_exercise, on='Date', how='outer')
# Display the final merged dataframe
final_data.tail(10)

,Session Start,Session End,Total Sleep Duration (hrs),Average HRV (ms),Date,Awake Duration (hrs),REM Duration (hrs),Core Duration (hrs),Deep Duration (hrs),Total Steps,Total Standing Hours,Total Basal Energy Burned (kcal),Total Active Energy Burned (kcal),Total Exercise Minutes
945,2024-06-13 02:06:46-07:00,2024-06-13 08:13:16-07:00,6.108333,21.5059,2024-06-13,0.450000,0.691667,4.408333,0.558333,37879.0,15.0,1755.987,912.462,64.0
946,NaT,NaT,NaN,NaN,2024-06-14,NaN,NaN,NaN,NaN,25315.0,14.0,1760.404,858.888,91.0
947,2024-06-15 01:52:55-07:00,2024-06-15 08:32:55-07:00,6.666667,46.7395,2024-06-15,1.000000,0.825000,3.375000,1.466667,30711.0,14.0,1714.536,675.773,57.0
948,NaT,NaT,NaN,NaN,2024-06-16,NaN,NaN,NaN,NaN,12454.0,13.0,1706.756,281.928,7.0
949,2024-06-17 01:30:03-07:00,2024-06-17 05:35:03-07:00,4.083333,27.3321,2024-06-17,0.208333,0.533333,2.666667,0.675000,19854.0,15.0,1700.459,643.673,53.0
950,NaT,NaT,NaN,NaN,2024-06-18,NaN,NaN,NaN,NaN,6426.0,NaN,594.77,2.867,1.0
951,NaT,NaT,NaN,NaN,2024-06-19,NaN,NaN,NaN,NaN,25375.0,9.0,932.314,534.205,77.0
952,NaT,NaT,NaN,NaN,2024-06-20,NaN,NaN,NaN,NaN,7348.0,8.0,1617.792,164.59,7.0
953,2024-06-21 01:36:38-07:00,2024-06-21 08:06:08-07:00,6.491667,39.1374,2024-06-21,0.150000,1.275000,3.700000,1.366667,10195.0,8.0,1191.799,277.496,21.0
954,NaT,NaT,NaN,NaN,2024-06-22,NaN,NaN,NaN,NaN,2547.0,NaN,NaN,NaN,NaN


In [54]:
april_summary = final_data[(final_data['Date'].dt.month == 4) & (final_data['Date'].dt.year == 2024)]


# Display the data for April
# april_summary

In [55]:
final_data.to_csv('merged_health_data_second.csv', index=False)